In [1]:
from utils.file_handling import *
from utils.misc import *
from utils.matplot_handlers import *
from utils.visualizers import *
from segmentation.composite_action_extractor import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

# Entropy Parameters
csv_file_name = 'jigsaw_Knot_Tying_B001_capture1'
analysis_method = 'w'

# Read in entropy csv
base_path = get_absolute_path('data', directory_file_hint=csv_file_name, ignore_files=False)
csv_path = os.path.join(base_path, 'state_action_data.csv')
main_df = pd.read_csv(csv_path, index_col=None)

In [ ]:
composite_actions, info = CompositeActionExtractor.get_composite_actions(dataframe=main_df, analysis_method=analysis_method, 
                                                            window_size=5)

In [ ]:
matplotlib.rcParams['figure.figsize'] = [15, 5]
plt.subplot(121)
plt.plot(info['state_analyzed_norm'])
plt.ylabel('Normalized / Windowed \n/ Measured Values')
plt.xlabel('Time Steps')
plt.title(f'State Measurements for {csv_file_name} EM')#using {analysis_method}')
plt.subplot(122)
plt.plot(info['action_analyzed_norm'])
plt.ylabel('Normalized / Windowed \n/ Measured Values')
plt.xlabel('Time Steps')
plt.title(f'Action Measurements for {csv_file_name} EM')#using {analysis_method}')

In [ ]:
matplotlib.rcParams['figure.figsize'] = [15, 5]
plt.subplot(121)
plt.plot(info['state_analyzed_binary'])
plt.ylabel('Normalized / Windowed \n/ Threshed / Measured Values')
plt.xlabel('Time Steps')
plt.title(f'State Threshed Measurements for \n{csv_file_name} using {analysis_method}')
plt.subplot(122)
plt.plot(info['action_analyzed_binary'])
plt.ylabel('Normalized / Windowed \n/ Threshed / Measured Values')
plt.xlabel('Time Steps')
plt.title(f'Action Threshed Measurements for \n{csv_file_name} using {analysis_method}')

In [ ]:
matplotlib.rcParams['figure.figsize'] = [8, 5]
for key in info['index_groups']:
    plt.subplot(1,len(info['index_groups']),1)
    plt.plot(info['index_groups'][key])
    plt.ylabel('Normalized / Windowed / \nThreshed / Measured Values')
    plt.xlabel('Time Steps')
    plt.title(f'Correlated Measurements for \n{csv_file_name} using {analysis_method}')

In [ ]:
matplotlib.rcParams['figure.figsize'] = [10, 5]
plt.plot(info['actions'])
for action in composite_actions:
    for i, segment in enumerate(composite_actions[action]):
        plt.plot(segment, label=f'composite Action {action} segment #{i}')

plt.title('Discovered Composite Actions')
plt.ylabel('Raw Action Values \n(Overlayed composite Action Segments)')
plt.xlabel('Time Steps')
# plt.legend(bbox_to_anchor=(1.01,1), loc="upper left")

In [ ]:
## We can take a look at primarily the action with the maximum total entropy
scored_composite_actions = {key: [(seg, info['state_analyzed_norm'][~np.isnan(seg)].max()) for seg in composite_actions[key]] for key in composite_actions}
k = 1
for key in scored_composite_actions:
    scored_composite_actions[key] = list(sorted(scored_composite_actions[key], key=lambda x: x[1], reverse=True))[:k]

In [ ]:
isolated_composite_actions = {key: [seg[~np.isnan(seg)] for seg in composite_actions[key]] for key in composite_actions}

In [ ]:
from ipywidgets import FloatSlider, IntSlider
from ipywidgets import interact, interactive, fixed, interact_manual

matplotlib.rcParams["figure.facecolor"] = 'white'
def f(i):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1,2,1)
    plt.imshow(video[i])
    plt.subplot(1,2,2)
    plt.subplots_adjust(right=1.5)
    plt.plot(info['index_groups'][0], markevery=[i], marker='|', markeredgecolor='r', 
         markersize=40, mew=4)
    plt.plot(scored_composite_actions[0][0][0])
    plt.ylabel('Normalized / Windowed / \nThreshed / Measured Values')
    plt.xlabel('Time Steps')
    plt.title(f'Highest Effective Correlated Action Joint 0 Measurements for \n{csv_file_name} using {analysis_method}')
    

interact(f,i=IntSlider(min=0, max=video.shape[0]-1, step=1));

In [ ]:
import pickle

name = os.path.join(base_path, 'segments')
if name[-4:] != ".pkl":
    name += ".pkl"
with open(name, 'wb') as f:
    pickle.dump(isolated_composite_actions, f, pickle.HIGHEST_PROTOCOL)